In [106]:
# ssh -L 9000:localhost:9000 thanrada@10.121.101.110
# http://127.0.0.1:9000/?token=3fa074da2984241394d534e672b246eff7b82102c9b6f317
# https://iservice.dopa.go.th/web/map/license/map/

In [107]:
import pandas as pd
import numpy as np
import subprocess

In [108]:
hotel = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/bora/hotel_bangkok.csv').iloc[:,[0,1,3,4]]
hotel.columns = ['Hotel_Ref','Hotel_Name','Hotel_Address','N']
hotel["Hotel_Address"]= hotel["Hotel_Address"].replace("บ้านเลขที่ ","", regex=True)
hotel.head()

,Hotel_Ref,Hotel_Name,Hotel_Address,N
0,101/2563,โรงแรม สุทธิสารพ้อยส์,45 ซ.ยิ้มอุปถัมภ์ แขวงรัชดาภิเษก เขตดินแดง จ.ก...,42
1,136/2563,โรงแรม คณาบูทีค 2,"3,5,7,9,11 ซ.จรัญสนิทวงศ์ 57/2 แขวงบางบำหรุ เข...",24
2,152/2563,วัลย์ลดา เพลส,71/1 ซ.รามคำแหง 53 (จันทร์ศรีชวาลา) แขวงพลับพล...,50
3,214/2563,โรงแรม นิวสยาม พาเลซวิลล์ 2,56 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48
4,215/2563,โรงแรม นิวสยาม พาเลซวิลล์ 3,58 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48


In [109]:
trip_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/OTA/trip_com_bkk.csv')[['name','name href','address']].dropna()
trip_com['OTA'] = 'trip_com'

booking_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/main/OTA/booking_com_bkk_with_addr.csv', engine='python')[['name href','name','address']].dropna()
booking_com['OTA'] = 'booking_com'

# รวมสอง DataFrame เข้าด้วยกัน
# OTA_df = pd.concat([trip_com, booking_com], ignore_index=True)
OTA_df = pd.concat([trip_com], ignore_index=True).sample(20,random_state=42)
OTA_df.head()

,name,name href,address,OTA
31,โรงแรมคิวว์ สุขุมวิท,https://th.trip.com/hotels/detail/?cityId=359&...,"9 สุขุมวิท 79 พระโขนงเหนือ, วัฒนา, เขตวัฒนา, ก...",trip_com
413,โรงแรมสำราญเพลส,https://th.trip.com/hotels/detail/?cityId=359&...,"302 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",trip_com
536,ไอบิส สไตล์ กรุงเทพ สุขุมวิท 4,https://th.trip.com/hotels/detail/?cityId=359&...,"27 ซ.สุขุมวิท 4 คลองเตย, เขตคลองเตย, กรุงเทพฯ,...",trip_com
745,ทู ทรี อะ โฮมลี่ โฮเทล,https://th.trip.com/hotels/detail/?cityId=359&...,"100 ซ. สุขุมวิท 23 แขวงคลองเตยเหนือ, เขตวัฒนา,...",trip_com
877,โรงแรมกลับ,https://th.trip.com/hotels/detail/?cityId=359&...,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com


## Method 1: Semantic search

### mrp/simcse-model-m-bert-thai-cased

In [110]:
# ---------- ตั้งค่า (from 614a1f16) ----------
import re, unicodedata
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1) Clean: ชื่อ/ที่อยู่ + สร้าง "คีย์" name+address
# =========================
def normalize_name(s: str) -> str:
    """ตัดคำ 'โรงแรม/Hotel' และ 'กรุงเทพ/กรุงเทพฯ/กรุงเทพมหานคร' + ลบเว้นวรรค/สัญลักษณ์"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower()
    s = re.sub(r"(โรงแรม|hotel| โฮเทล|โฮเทล)", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"กรุงเทพ(?:มหานคร|ฯ)?", " ", s)  # ตัดคำ 'กรุงเทพ' ออกจาก 'ชื่อ'
    s = re.sub(r"[,\|\(\)\[\]{}\"'’`•·/.:;#&\-–—_+*^@!$?~]", " ", s)
    s = re.sub(r"\s+", "", s)  # ลบช่องว่างทั้งหมด
    return s

def normalize_addr(s: str) -> str:
    """คงโครงสร้างที่อยู่ไว้ (ไม่ลบช่องว่างทั้งหมด) แต่ normalize ให้เท่า ๆ กัน"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower().strip()

    # ทำให้คำหลักสอดคล้องกัน
    rep = {
        "กทม.": "กรุงเทพมหานคร",
        "กรุงเทพฯ": "กรุงเทพมหานคร",
        "จ.": "",              # จ.กรุงเทพมหานคร -> กรุงเทพมหานคร
        "เขต ": "",
        "แขวง ": "",
        "ถ.": "",
        "ถนน ": "",        # กันกรณีมีช่องว่างเพี้ยน
        "บ้านเลขที่": "",
    }
    for k,v in rep.items():
        s = s.replace(k, v)

    # เก็บตัวอักษร/ตัวเลข/เว้นวรรคและเครื่องหมาย / , - บางส่วน
    s = re.sub(r"[\"'’`•·\|\[\]{}#@!$^*?_+=:;,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return "".join(s.split())

# apply clean
OTA_df["Trip_Name_n"]    = OTA_df["name"].map(normalize_name)
OTA_df["Trip_Address_n"] = OTA_df["address"].map(normalize_addr)
hotel["Hotel_Name_n"]    = hotel["Hotel_Name"].map(normalize_name)
hotel["Hotel_Address_n"] = hotel["Hotel_Address"].map(normalize_addr)

# คีย์ค้นหา = name + address (ตามที่ต้องการ)
OTA_df["Trip_Key"]   = (OTA_df["Trip_Name_n"]   + " " + OTA_df["Trip_Address_n"]).str.strip()
hotel["Hotel_Key"] = (hotel["Hotel_Name_n"] + " " + hotel["Hotel_Address_n"]).str.strip()

In [111]:
OTA_df[OTA_df['Trip_Name_n'].str.contains('กลับ', na=False)]

,name,name href,address,OTA,Trip_Name_n,Trip_Address_n,Trip_Key
877,โรงแรมกลับ,https://th.trip.com/hotels/detail/?cityId=359&...,"22/1 ถ.เพชรบุรี ซ.13, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,กลับ,22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400,กลับ 22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400


In [112]:
hotel[hotel['Hotel_Name'].str.contains('กลับ', na=False)]

,Hotel_Ref,Hotel_Name,Hotel_Address,N,Hotel_Name_n,Hotel_Address_n,Hotel_Key
450,141/2565,กลับโฮเทล,22/1 ซ.สมประสงค์ 2 แขวงถนนพญาไท เขตราชเทวี จ.ก...,79,กลับ,22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุงเทพม...,กลับ 22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุ...


In [113]:
# 0) ONE SOURCE OF TRUTH
OTA_df = OTA_df.reset_index(drop=True).copy()
hotel  = hotel.reset_index(drop=True).copy()

# 1) ensure columns exist & fillna
for col in ["Trip_Name_n","Trip_Address_n"]:
    if col not in OTA_df.columns:
        raise ValueError(f"ไม่พบคอลัมน์ {col} ใน OTA_df — ต้อง normalize ให้เรียบร้อยก่อน")
for col in ["Hotel_Name_n","Hotel_Address_n"]:
    if col not in hotel.columns:
        raise ValueError(f"ไม่พบคอลัมน์ {col} ใน hotel — ต้อง normalize ให้เรียบร้อยก่อน")

OTA_df["Trip_Name_n"]    = OTA_df["Trip_Name_n"].fillna("").astype(str)
OTA_df["Trip_Address_n"] = OTA_df["Trip_Address_n"].fillna("").astype(str)
hotel["Hotel_Name_n"]    = hotel["Hotel_Name_n"].fillna("").astype(str)
hotel["Hotel_Address_n"] = hotel["Hotel_Address_n"].fillna("").astype(str)

# 2) สร้างคีย์รวม (ถ้ายังไม่มี/อยากทับ)
OTA_df["Trip_Key"] = (OTA_df["Trip_Name_n"] + " " + OTA_df["Trip_Address_n"]).str.strip()
hotel["Hotel_Key"] = (hotel["Hotel_Name_n"] + " " + hotel["Hotel_Address_n"]).str.strip()

# 3) โหลดโมเดล (ถ้ายังไม่มี)
MODEL = "mrp/simcse-model-m-bert-thai-cased"
model = SentenceTransformer(MODEL)

# 4) Encode ใหม่ให้ “สอดคล้องกับ DataFrame นี้เท่านั้น”
emb_trip_name  = model.encode(OTA_df["Trip_Name_n"].tolist(),    convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_addr  = model.encode(OTA_df["Trip_Address_n"].tolist(),  convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_key   = model.encode(OTA_df["Trip_Key"].tolist(),        convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

emb_hotel_name = model.encode(hotel["Hotel_Name_n"].tolist(),     convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_addr = model.encode(hotel["Hotel_Address_n"].tolist(),   convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_key  = model.encode(hotel["Hotel_Key"].tolist(),         convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

# 5) ตรวจ alignment
assert emb_trip_key.shape[0]  == len(OTA_df), f"trip mismatch: emb={emb_trip_key.shape[0]} vs df={len(OTA_df)}"
assert emb_hotel_key.shape[0] == len(hotel),  f"hotel mismatch: emb={emb_hotel_key.shape[0]} vs df={len(hotel)}"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

# Vector database

In [114]:
!pip install langchain langchain-community
!pip install faiss-cpu

## 1) สร้าง Vector DB จาก Hotel_Key

In [115]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document


def create_hotel_vector_db(hotel_df: pd.DataFrame, save_path: str):
    docs = []
    for _, row in hotel_df.iterrows():
        content = str(row["Hotel_Key"])
        metadata = row.to_dict()
        docs.append(Document(page_content=content, metadata=metadata))

    embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-base",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )

    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local(save_path)
    return vectorstore


def load_hotel_vector_db(save_path: str):
    embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-base",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )
    vectorstore = FAISS.load_local(
        save_path, embeddings, allow_dangerous_deserialization=True
    )
    return vectorstore


In [116]:
# ครั้งแรก
# vectorstore = create_hotel_vector_db(hotel, "faiss_hotel_db")

# ครั้งถัดไป
vectorstore = load_hotel_vector_db("faiss_hotel_db")


## 2 ฟังก์ชันให้ LLM ตัดสิน True/False: OTA_Key vs Hotel_Key

In [117]:
import re
from langchain_community.llms import Ollama

In [118]:
def llm_confirm_same_hotel(entity1: str, entity2: str) -> bool:
    llm = Ollama(
        model="scb10x/llama3.1-typhoon2-8b-instruct",
        base_url="http://127.0.0.1:11434")

    prompt = f"""
คุณคือผู้เชี่ยวชาญด้านโรงแรมและที่พักในประเทศไทย
หน้าที่ของคุณคือตัดสินว่าสองสตริงนี้อ้างถึงโรงแรม/ที่พักเดียวกันหรือไม่
แต่ละสตริงจะขึ้นต้นด้วย ชื่อโรงแรมตามด้วยสถานที่ เช่น สำราญเพลส [ชื่อ] 302 เขตราชเทวี กรุงเทพมหานคร 10400 [สถานที่]

กติกา:
1. เปรียบเทียบเฉพาะ 2 สถานที่ที่ให้เท่านั้น
2. ถ้าคุณไม่มั่นใจ หรือข้อมูลไม่พอ ให้ตอบ False
3. ให้ตอบเพียงคำเดียวเท่านั้น: True หรือ False

ตัวอย่าง:

[ตัวอย่าง 1 - True] (เหมือนทั้งชื่อทั้งที่อยู่)
Entity 1: สำราญเพลส 302 เขตราชเทวี กรุงเทพมหานคร 10400
Entity 2: สำราญเพลส 302 แขวงถนนเพชรบุรี เขตราชเทวี กรุงเทพมหานคร 10400
Answer: True

[ตัวอย่าง 2 - False] (ที่อยู่คล้ายกันแต่ละที่แน่นอน)
Entity 1: อีเลฟเว่นแบงคอกสุขุมวิท11 11 ซอยสุขุมวิท 11 เขตวัฒนา กรุงเทพมหานคร 10110
Entity 2: เทรเวลลอดจ์สุขุมวิท11 30/9-10 ซอยสุขุมวิท 11 (ไชยยศ) คลองเตยเหนือ วัฒนา กรุงเทพมหานคร 10110
Answer: False

[ตัวอย่าง 3 - False (ชื่อคล้าย แต่อยู่คนละย่าน)]
Entity 1: ทูทรีอะโฮมลี่ 100 ซอยสุขุมวิท 23 คลองเตยเหนือ วัฒนา กรุงเทพมหานคร 10110
Entity 2: ทูทรีอะโฮมลี่ 55 ซอยลาดพร้าว 10 จตุจักร กรุงเทพมหานคร 10900
Answer: False

[ตัวอย่าง 4 - True (สะกด/เว้นวรรคต่างเล็กน้อย รหัสไปรษณีย์ต่างเล็กน้อย แต่เหมือนกันทั้งชื่อและที่อยู่)]
Entity 1: ทูทรีอะโฮมลี่ 100 ซอยสุขุมวิท 23 คลองเตยเหนือ วัฒนา กรุงเทพมหานคร 10110
Entity 2: ทู ทรี อะ โฮมลี่ 100 สุขุมวิท 23 คลองเตยเหนือ วัฒนา กรุงเทพมหานคร 10111
Answer: True

[ตัวอย่าง 5 - False (ชื่อคนละโรงแรม แม้อยู่ซอยเดียวกัน)]
Entity 1: ไอบิสริเวอร์ไซด์ 27 ซ.เจริญนคร 17 เจริญนคร แขวงบางลําภูล่าง เขตคลองสาน กรุงเทพมหานคร 10600
Entity 2: เดอะโคซี่ริเวอร์ 1 ซอยเจริญนคร 17 แขวงบางลําภูล่าง เขตคลองสาน กรุงเทพมหานคร 10600
Answer: False

ตอนนี้ให้ตัดสินคู่ต่อไป:

Entity 1: {entity1}
Entity 2: {entity2}
Answer:
"""

    raw = llm.predict(prompt).strip().lower()

    m = re.search(r"\b(true|false)\b", raw)
    if not m:
        return False
    return m.group(1) == "true"


In [119]:
def match_ota_to_hotels(ota_df: pd.DataFrame, vectorstore, top_k: int = 5):
    results = ota_df.copy()

    match_found = []
    match_ref = []
    match_name = []
    match_addr = []
    match_key = []

    for _, row in ota_df.iterrows():
        query = str(row["Trip_Key"]).strip()

        candidates = vectorstore.similarity_search(query, k=top_k)

        found = False
        h_ref = h_name = h_addr = h_key = None

        for doc in candidates:
            hotel_key = doc.page_content
            if llm_confirm_same_hotel(query, hotel_key):
                found = True
                h_ref = doc.metadata.get("Hotel_Ref")
                h_name = doc.metadata.get("Hotel_Name")
                h_addr = doc.metadata.get("Hotel_Address")
                h_key = doc.metadata.get("Hotel_Key")
                break

        match_found.append(found)
        match_ref.append(h_ref)
        match_name.append(h_name)
        match_addr.append(h_addr)
        match_key.append(h_key)

    results["match_found"] = match_found
    results["matched_Hotel_Ref"] = match_ref
    results["matched_Hotel_Name"] = match_name
    results["matched_Hotel_Address"] = match_addr
    results["matched_Hotel_Key"] = match_key

    return results


## 3) ฟังก์ชันหลัก: match OTA vs Hotel DB
* ใช้ OTA_Key เป็น query
* หา top-k ผู้เข้ารอบ
* ให้ LLM confirm
* ถ้า match → เก็บข้อมูลโรงแรม
* ถ้าไม่ match เลย → ใส่ False

In [123]:
import re, os
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from tqdm.auto import tqdm
from langchain_community.llms import Ollama


# ------------------ LLM: สร้างครั้งเดียว ใช้ซ้ำ ------------------

@lru_cache(maxsize=1)
def get_llm():
    return Ollama(
        model="scb10x/llama3.1-typhoon2-8b-instruct",
        # model="qwen2.5:14b-instruct",
        base_url="http://127.0.0.1:11434",
    )


    prompt = f"""
คุณเป็นผู้ตัดสินว่าโรงแรม 2 แห่งในประเทศไทยเป็นที่เดียวกันหรือไม่

หน้าที่ของคุณ:
- พิจารณาจากชื่อโรงแรมและรายละเอียดที่อยู่ทั้งหมดอย่างมีเหตุผล
- เปรียบเทียบองค์ประกอบต่าง ๆ เช่น ชื่อ ซอย ถนน แขวง/ตำบล เขต/อำเภอ จังหวัด และรหัสไปรษณีย์
- ถ้าองค์ประกอบโดยรวมบ่งชี้ไปในทิศทางเดียวกันว่าเป็นสถานที่เดียวกัน แม้ชื่อจะสะกดต่างกันเล็กน้อย หรือรายละเอียดที่อยู่เขียนต่างรูปแบบกัน ให้เอนเอียงไปทาง True
- ถ้าองค์ประกอบสำคัญของที่อยู่ต่างกันชัดเจน เช่น คนละเขต คนละอำเภอ หรือคนละจังหวัด ให้เอนเอียงไปทาง False
- หากข้อมูลคลุมเครือหรือไม่เพียงพอในการตัดสินอย่างมั่นใจ ให้ตอบ False

"""


    raw = get_llm().predict(prompt).strip().lower()
    m = re.search(r"\b(true|false)\b", raw)
    return m.group(1) == "true" if m else False


# ------------------ worker สำหรับแต่ละแถว (ใช้ Trip_Key เดิม) ------------------

def process_row(i: int, row: pd.Series, vectorstore, top_k: int = 1):
    trip_key = str(row.get("Trip_Key", "") or "").strip()

    # default return ถ้าไม่มี Trip_Key
    if not trip_key:
        return (
            i,
            False,
            None,  # matched_Hotel_Ref
            None,  # matched_Hotel_Name
            None,  # matched_Hotel_Address
            None,  # matched_Hotel_Key
            None,  # top1_doc_content
            None,  # top1_faiss_score
        )

    # ใช้ Trip_Key เป็น query ไปหาใน vectorstore (top_k = 1) + score
    try:
        docs_with_scores = vectorstore.similarity_search_with_score(trip_key, k=top_k)
    except Exception:
        docs_with_scores = []

    if not docs_with_scores:
        return (
            i,
            False,
            None,
            None,
            None,
            None,
            None,
            None,
        )

    # เราใช้แค่ตัวที่ใกล้สุดตัวเดียว
    doc, score = docs_with_scores[0]
    hotel_key = doc.page_content  # ควรเป็น "ชื่อ + ที่อยู่" ของโรงแรม reference

    # เก็บ TOP1 doc + score ไว้เลย (ก่อน LLM confirm)
    top1_doc_content = hotel_key
    top1_faiss_score = float(score) if score is not None else None

    # ให้ LLM confirm ว่าใช่โรงแรมเดียวกันไหม
    if llm_confirm_same_hotel(trip_key, hotel_key):
        md = doc.metadata
        return (
            i,
            True,
            md.get("Hotel_Ref"),
            md.get("Hotel_Name"),
            md.get("Hotel_Address"),
            md.get("Hotel_Key"),
            top1_doc_content,
            top1_faiss_score,
        )

    # กรณี LLM บอกว่าไม่ใช่ ก็ยังเก็บ TOP1 + score ให้ดูได้
    return (
        i,
        False,
        None,
        None,
        None,
        None,
        top1_doc_content,
        top1_faiss_score,
    )


# ------------------ main function: ใช้ 20 core + progress bar ------------------

def match_ota_to_hotels(df: pd.DataFrame, vectorstore, top_k: int = 1) -> pd.DataFrame:
    if df.empty:
        return df.copy()

    n = len(df)
    out = {
        "match_found": [False] * n,
        "matched_Hotel_Ref": [None] * n,
        "matched_Hotel_Name": [None] * n,
        "matched_Hotel_Address": [None] * n,
        "matched_Hotel_Key": [None] * n,
        "top1_doc_content": [None] * n,   # ใหม่: เก็บเนื้อหา doc ที่ใกล้สุด
        "top1_faiss_score": [None] * n,   # ใหม่: เก็บ score จาก FAISS
    }

    # ใช้ 20 core ตามที่บอกมา (ตอนนี้ใส่ตัวอย่างเป็น 20)
    workers = 5

    with ThreadPoolExecutor(max_workers=workers) as exe:
        futures = {
            exe.submit(process_row, i, row, vectorstore, top_k): i
            for i, row in df.iterrows()
        }

        for fut in tqdm(as_completed(futures), total=len(futures), desc="Matching OTA"):
            (
                idx,
                found,
                r,
                n_name,
                a,
                k,
                top1_content,
                top1_score,
            ) = fut.result()

            out["match_found"][idx] = found
            out["matched_Hotel_Ref"][idx] = r
            out["matched_Hotel_Name"][idx] = n_name
            out["matched_Hotel_Address"][idx] = a
            out["matched_Hotel_Key"][idx] = k
            out["top1_doc_content"][idx] = top1_content
            out["top1_faiss_score"][idx] = top1_score

    result = df.copy()
    for col, vals in out.items():
        result[col] = vals
    return result


In [124]:
# สมมติ df = OTA_df และสร้าง vectorstore ไว้แล้ว
matched_df = match_ota_to_hotels(OTA_df, vectorstore, top_k=1)


Matching OTA:   0%|          | 0/20 [00:00<?, ?it/s]

In [125]:
matched_df[['Trip_Key','match_found','top1_doc_content','top1_faiss_score']]

,Trip_Key,match_found,top1_doc_content,top1_faiss_score
0,คิวว์สุขุมวิท 9สุขุมวิท79พระโขนงเหนือวัฒนาเขตว...,True,คิวว์สุขุมวิท 9ซ.สุขุมวิท79(นิยม)แขวงพระโขนงเห...,0.041817
1,สําราญเพลส 302เขตราชเทวีกรุงเทพมหานคร10400,True,สําราญเพลส 302เพชรบุรีแขวงถนนเพชรบุรีเขตราชเทว...,0.046297
2,ไอบิสสไตล์สุขุมวิท4 27ซ.สุขุมวิท4คลองเตยเขตคลอ...,True,โนโวเทลไอบิสสไตล์สุขุมวิท4 27ซ.สุขุมวิท4แขวงคล...,0.048400
3,ทูทรีอะโฮมลี่ 100ซ.สุขุมวิท23แขวงคลองเตยเหนือเ...,False,ทูทรี 100ซ.สุขุมวิท23(ประสานมิตร)แขวงคลองเตยเห...,0.079489
4,กลับ 22/1เพชรบุรีซ.13เขตราชเทวีกรุงเทพมหานคร10400,True,กลับ 22/1ซ.สมประสงค์2แขวงถนนพญาไทเขตราชเทวีกรุ...,0.085076
5,ดั้งเดิมอินเดอะพาร์คข้าวสาร 68-70ข้าวสารพระนคร...,True,ข้าวสารพาเลซอินน์ 139ข้าวสารแขวงตลาดยอดเขตพระน...,0.149464
6,อัลท์นานาบายยูเอชจี 222ซ.สุขุมวิท4แยก2(นานา)สุ...,True,อัลท์นานา 222ซ.สุขุมวิท4แขวงคลองเตยเขตคลองเตยก...,0.074492
7,ชานบางกอกน้อย 30/1สมเด็จพระปิ่นเกล้าเขตบางกอกน...,True,ชานบางกอกน้อย 30/1ซ.ริมคลองบางกอกน้อยแขวงอรุณอ...,0.104578
8,เอสทรีสยาม 2/6ซ.เกษมสันต์3พระราม1แขวงวังใหม่เข...,True,เอสทรีสยาม 2/6ซ.เกษมสันต์3แขวงวังใหม่เขตปทุมวั...,0.040911
9,บีอาร์บีโฮสเทลสีลม 5/8-9ซ.สีลม3(ซ.พิพัฒน์)สีลม...,False,บ้านสีลมซอย3 16/11ซ.พิพัฒน์แขวงสีลมเขตบางรักกร...,0.133717
